In [14]:
import os
import subprocess

## directories
user_path = os.path.expanduser('~')

if "mrung" in user_path : 
    exe_dir = os.path.join(user_path, 'Box/NU-malaria-team/projects/binaries/compartments/')
    git_dir = os.path.join(user_path, 'gitrepos/covid-chicago/')
elif 'geickelb1' in user_path:
    exe_dir = os.path.join(user_path,'Desktop/compartments/')
    git_dir = os.path.join(user_path, 'Documents/Github/covid-chicago/')

# Selected range values from SEIR Parameter Estimates.xlsx
# Need to update to run for sample distributions, rather than discrete values
initial_infect = [1,5,10]
Ki = [0.0009, 0.05, 0.32]
incubation_pd = [6.63, 4.2, 12.4]
recovery_rate = [6,13, 16 ]

In [ ]:
; simplemodel

(import (rnrs) (emodl cmslib))

(start-model "seir.emodl")

#Illinois specific numbers. ie 12M people, with 12 infected. 
## chemical species that ar possible
(species S 12671809)
(species E)
(species I 12)
(species R)


In [32]:
### initializing some default parameters and n for counties:
county_dic={'illinois':[12671890, 0, 12, 0]} #might need to change the 0's

param_dic={'ki':0.00000019,'incubation_pd':6.63, 'recovery_rate':16, 'waning':180}

In [26]:
#initial SEIR numbers:
def SIER_init(county_dic, county):
    "## chemical species starting values"
    
    S= str("(species S_{} {})".format(county, county_dic[county][0]))
    E= str("(species E_{} {})".format(county, county_dic[county][1]))
    I= str("(species I_{} {})".format(county, county_dic[county][2]))
    R= str("(species R_{} {})".format(county, county_dic[county][3]))
    
    return(S,E,I,R)

In [30]:
##testing
# S_i, E_i, I_i, R_i = SIER_init(county_dic, 'illinois')
# print(S_i)

print(SIER_init(county_dic, 'illinois'))

('(species S_illinois 12671890)', '(species E_illinois 0)', '(species I_illinois 12)', '(species R_illinois 0)')


In [20]:
def SIER_output(county):
    "## chemical species that are possible (what i want reporeted in output file)"
    S= str("(observe susceptible_{} S_{})".format(county, county))
    E= str("(observe exposed_{} E_{})".format(county, county))
    I= str("(observe infectious_{} I_{})".format(county, county))
    R= str("(observe recovered_{} R_{})".format(county, county))
    
    return(S,E,I,R)

In [57]:
SIER_output('illinois')

('(observe susceptible_illinois S_illinois)',
 '(observe exposed_illinois E_illinois)',
 '(observe infectious_illinois I_illinois)',
 '(observe recovered_illinois R_illinois)')

In [100]:
def params(param_dic):
    ki=param_dic['ki']
    params_string = """
(param Ki {})
(param incubation_pd {})
(param Kl (/ 1 incubation_pd))
(param recovery_rate {})
(param Kr (/ 1 recovery_rate))
(param waning {})
(param Kw (/ 1 waning))
    """.format(param_dic['ki'],
               param_dic['incubation_pd'],
               param_dic['recovery_rate'],
               param_dic['waning']
              )

    return(params_string)

In [90]:
params(param_dic)

'(param Ki 1.9e-07)\n(param incubation_pd 6.63)\n(param Kl (/ 1 incubation_pd))\n(param recovery_rate 16)\n(param Kr (/ 1 recovery_rate))\n(param waning 180)\n(param Kw (/ 1 waning))\n    '

In [113]:
def reactions(county):
    county= str(county)
    
    reaction_str="""
(reaction exposure   (S_{}) (E_{}) (* Ki S_{} I_{}))
(reaction infection  (E_{})   (I_{})   (* Kl E_{}))
(reaction recovery   (I_{})   (R_{})   (* Kr I_{}))
;(reaction waning     (R_{})   (S_{})   (* Kw R_{}))

""".format(county, county, county,
            county, county, county,
            county, county, county,
            county, county, county,
            county)
    return(reaction_str)


In [73]:
reactions('illinois')

'(reaction exposure   (S_illinois) (E_illinois) (* Ki S_illinois I_illinois))\n    (reaction infection  (E_illinois)   (I_illinois)   (* Kl E_illinois))\n    (reaction recovery   (I_illinois)   (R_illinois)   (* Kr I_illinois))\n    ;(reaction waning     (R_illinois)   (S_illinois)   (* Kw R_illinois))\n    \n    '

In [102]:
model_name= "seir.emodl"

header_str="""
; simplemodel

(import (rnrs) (emodl cmslib))

(start-model "{}")

""".format(model_name)


footer_str ="(end-model)"

In [54]:
total_string=""

In [55]:
total_string+ header_str

'\n; simplemodel\n\n(import (rnrs) (emodl cmslib))\n\n(start-model "seir.emodl")\n\n'

In [114]:
#working towards assembling the main
total_string=""
total_string= total_string+ header_str

for key in county_dic.keys():
    S,E,I,R =SIER_init(county_dic, key)
    total_string= total_string+ S + '\n' + E + '\n' + I + '\n' + R + '\n'
    
    S,E,I,R =SIER_output(key)
    total_string= total_string+ S + '\n' + E + '\n' + I + '\n' + R + '\n'
    
param= params(param_dic)
total_string= total_string+ param

for key in county_dic.keys(): 
    reaction= reactions(key)
    total_string= total_string+ reaction
total_string=total_string+footer_str
    

print(total_string)
    


; simplemodel

(import (rnrs) (emodl cmslib))

(start-model "seir.emodl")

(species S_illinois 12671890)
(species E_illinois 0)
(species I_illinois 12)
(species R_illinois 0)
(observe susceptible_illinois S_illinois)
(observe exposed_illinois E_illinois)
(observe infectious_illinois I_illinois)
(observe recovered_illinois R_illinois)

(param Ki 1.9e-07)
(param incubation_pd 6.63)
(param Kl (/ 1 incubation_pd))
(param recovery_rate 16)
(param Kr (/ 1 recovery_rate))
(param waning 180)
(param Kw (/ 1 waning))
    
(reaction exposure   (S_illinois) (E_illinois) (* Ki S_illinois I_illinois))
(reaction infection  (E_illinois)   (I_illinois)   (* Kl E_illinois))
(reaction recovery   (I_illinois)   (R_illinois)   (* Kr I_illinois))
;(reaction waning     (R_illinois)   (S_illinois)   (* Kw R_illinois))

(end-model)


In [115]:
fin = open("GE_generated_test.emodl", "w")
#text_file = open("sample.txt", "w")
fin.write(total_string)
fin.close()


In [ ]:
    for i in enumerate(param) :
        print(i)
        fin = open("simplemodel_covid.emodl", "rt")
        data = fin.read()
        if(paramname ==  "initial_infect") : data = data.replace('(species I 10)', '(species I ' + str(i[1]) +')')
        if (paramname == "Ki") : data = data.replace('(param Ki 0.319)', '(param Ki '  + str(i[1]) +')')
        if (paramname == "incubation_pd") : data = data.replace('(param incubation_pd 6.63)', '(param incubation_pd ' + str(i[1]) +')')
        if (paramname == "recovery_rate") :data = data.replace('(param recovery_rate 16)', '(param recovery_rate '  + str(i[1]) +')')
        fin.close()
        fin = open("simplemodel_covid_i.emodl", "wt")
        fin.write(data)
        fin.close()
        # adjust simplemodel.cfg file as well
        fin = open("simplemodel.cfg", "rt")
        data_cfg = fin.read()
        data_cfg = data_cfg.replace('trajectories', 'trajectories_' + paramname + '_' + str(i[1]) )
        fin.close()
        fin = open("simplemodel_i.cfg", "wt")
        fin.write(data_cfg)
        fin.close()
        file = open('runModel_i.bat', 'w')
        file.write('\n"' + os.path.join(exe_dir, "compartments.exe") + '"' + ' -c ' + '"' + os.path.join(git_dir, "simplemodel_i.cfg") +
                   '"' + ' -m ' + '"' + os.path.join( git_dir, "simplemodel_covid_i.emodl", ) + '"')
        file.close()
        subprocess.call([r'runModel_i.bat'])

In [64]:
for key in county_dic.keys(): 
    print(str(key))

illinois


In [ ]:
; simplemodel

(import (rnrs) (emodl cmslib))

(start-model "seir.emodl")

#Illinois specific numbers. ie 12M people, with 12 infected. 
## chemical species that ar possible
(species S 12671809)
(species E)
(species I 12)
(species R)

## chemical species that ar possible (what i want reporeted in output file)
(observe susceptible S)
(observe exposed     E)
(observe infectious  I)
(observe recovered   R)

## chemical species that ar possible. population= sum(s,e,i,r)
### ;=comment out
;(observe population  (sum S E I R))

## chemical species that ar possible; these are the constants (tho can also be time varying if defined as fxn)
### note: this will not change based on # of counties. 
(param Ki 0.00000019)
(param incubation_pd 6.63)
(param Kl (/ 1 incubation_pd))
(param recovery_rate 16)
(param Kr (/ 1 recovery_rate))
(param waning 180)
(param Kw (/ 1 waning))

### the emodel itself could specify all of the counties. this would make the file ~600 lines long, ie SEIR for each county
### there is an idea of locals. 
### maybe write a function for all of this and run that function on each county?? look into files to see if possible. 
## these are the actual chemical reactions that are taking place in the system. 
(reaction exposure   (S) (E) (* Ki S I))  ## S->E at rate~ (Ki, s, i)
(reaction infection  (E)   (I)   (* Kl E))
(reaction recovery   (I)   (R)   (* Kr I))
; (reaction waning     (R)   (S)   (* Kw R))

(end-model)

In [ ]:


def runExp_singleParamChange(param, paramname ) :

    # param = Ki
    # paramname = "Ki"
    for i in enumerate(param) :
        print(i)
        fin = open("simplemodel_covid.emodl", "rt")
        data = fin.read()
        if(paramname ==  "initial_infect") : data = data.replace('(species I 10)', '(species I ' + str(i[1]) +')')
        if (paramname == "Ki") : data = data.replace('(param Ki 0.319)', '(param Ki '  + str(i[1]) +')')
        if (paramname == "incubation_pd") : data = data.replace('(param incubation_pd 6.63)', '(param incubation_pd ' + str(i[1]) +')')
        if (paramname == "recovery_rate") :data = data.replace('(param recovery_rate 16)', '(param recovery_rate '  + str(i[1]) +')')
        fin.close()
        fin = open("simplemodel_covid_i.emodl", "wt")
        fin.write(data)
        fin.close()
        # adjust simplemodel.cfg file as well
        fin = open("simplemodel.cfg", "rt")
        data_cfg = fin.read()
        data_cfg = data_cfg.replace('trajectories', 'trajectories_' + paramname + '_' + str(i[1]) )
        fin.close()
        fin = open("simplemodel_i.cfg", "wt")
        fin.write(data_cfg)
        fin.close()
        file = open('runModel_i.bat', 'w')
        file.write('\n"' + os.path.join(exe_dir, "compartments.exe") + '"' + ' -c ' + '"' + os.path.join(git_dir, "simplemodel_i.cfg") +
                   '"' + ' -m ' + '"' + os.path.join( git_dir, "simplemodel_covid_i.emodl", ) + '"')
        file.close()
        subprocess.call([r'runModel_i.bat'])


In [ ]:
runExp_singleParamChange(initial_infect,"initial_infect" )
runExp_singleParamChange(Ki ,"Ki " )
runExp_singleParamChange(incubation_pd,"incubation_pd" )
runExp_singleParamChange(recovery_rate,"recovery_rate" )